# Table of Contents

* [Introduction](#1)
* [Workspace Preparation](#2)
* [Data Preparation and Features Processing](#3)
* [Getting x_train, x_test, y_train, y_test](#4)
* [MLFlow workspace preparation and Use](#5)
* [Conclusions](#6)
* [References](#7)

# Introduction

> Nota: Este Jupyter Notebook en la mayor medida esta en Ingles. Se encuentra en español las partes del contexto y habra comentarios con spanglish. ;)

### Escenario del Notebook

- Una empresa activa en Big Data y Data Science desea contratar científicos de datos entre las personas que aprueben exitosamente algunos cursos impartidos por la empresa. Muchas personas se inscriben en su capacitación. La empresa quiere saber cuáles de estos candidatos realmente desean trabajar para la empresa después de la capacitación o si están buscando un nuevo empleo, ya que esto ayuda a reducir los costos y el tiempo, así como la calidad de la capacitación o la planificación de los cursos y la categorización de los candidatos. La información relacionada con la demografía, la educación y la experiencia está disponible a partir de la inscripción y matrícula de los candidatos.

- Este conjunto de datos está diseñado para comprender los factores que llevan a una persona a dejar su trabajo actual, también para investigaciones de recursos humanos. Utilizando el modelo o modelos que utilicen las credenciales actuales, la demografía y los datos de experiencia, **se predecirá la probabilidad de que un candidato busque un nuevo empleo o trabaje para la empresa, además de interpretar los factores que afectan la decisión del empleado.** 

- Todos los datos se dividen en conjuntos de entrenamiento y prueba. 

- El objetivo no está incluido en la prueba, pero se cuenta con el archivo de datos de los valores objetivo de prueba para tareas relacionadas. 

- También se proporciona una muestra de envío correspondiente al identificador de inscrito en el conjunto de prueba con las columnas: enrollee_id, target.


# Workspace preparation

In [2]:
from pathlib import Path
from urllib.parse import urlparse

import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                             roc_auc_score, classification_report)
import mlflow
import mlflow.sklearn

## Part2: Loading model from artifact to use

# Conclusions

# References

# Other key functions and Info